# Auto-Register %%agent Magic for Jupyter

This notebook creates a startup script that automatically loads the `%%agent` magic command every time you start Jupyter/IPython. The script will:

1. Locate the IPython directory in a platform-agnostic way (works on Windows, macOS, and Linux)
2. Check if the IPython directory exists, and create it if needed
3. Check if the `profile_default` directory exists inside the IPython directory, and create it if needed
4. Check if the `startup` directory exists inside `profile_default`, and create it if needed
5. Create a Python startup script that loads the `%%agent` magic extension
6. Save the script to the startup directory so it runs automatically when Jupyter/IPython starts

## Requirements
- jupyter_ai_agent_magics package must be installed

In [5]:
import os
import sys
from pathlib import Path

# Try to use IPython's built-in functions to locate the correct directory
try:
    # Import IPython's paths module which handles platform-specific paths
    from IPython.paths import get_ipython_dir
    ipython_dir = get_ipython_dir()
    print(f"Using IPython's get_ipython_dir(): {ipython_dir}")
except ImportError:
    # Fallback if IPython is not available or if we can't import the module
    print("Could not import IPython.paths, using fallback method")
    if sys.platform == 'win32':
        # On Windows, typically in %USERPROFILE%\.ipython
        ipython_dir = os.path.join(os.path.expanduser('~'), '.ipython')
    else:
        # On Unix systems (Linux, macOS)
        ipython_dir = os.path.expanduser('~/.ipython')
    print(f"Using fallback path: {ipython_dir}")

# Create the IPython directory if it doesn't exist
if not os.path.exists(ipython_dir):
    os.makedirs(ipython_dir)
    print(f"Created IPython directory: {ipython_dir}")
else:
    print(f"IPython directory already exists: {ipython_dir}")

# Create profile_default directory if it doesn't exist
profile_dir = os.path.join(ipython_dir, 'profile_default')
if not os.path.exists(profile_dir):
    os.makedirs(profile_dir)
    print(f"Created profile directory: {profile_dir}")
else:
    print(f"Profile directory already exists: {profile_dir}")

# Create startup directory if it doesn't exist
startup_dir = os.path.join(profile_dir, 'startup')
if not os.path.exists(startup_dir):
    os.makedirs(startup_dir)
    print(f"Created startup directory: {startup_dir}")
else:
    print(f"Startup directory already exists: {startup_dir}")

# Display the paths
print(f"\nIPython directory: {ipython_dir}")
print(f"Profile directory: {profile_dir}")
print(f"Startup directory: {startup_dir}")

Using IPython's get_ipython_dir(): /Users/alekseyv/.ipython
IPython directory already exists: /Users/alekseyv/.ipython
Profile directory already exists: /Users/alekseyv/.ipython/profile_default
Startup directory already exists: /Users/alekseyv/.ipython/profile_default/startup

IPython directory: /Users/alekseyv/.ipython
Profile directory: /Users/alekseyv/.ipython/profile_default
Startup directory: /Users/alekseyv/.ipython/profile_default/startup


In [6]:
# Create the startup script content
startup_script = '''# Auto-register %%agent magic
try:
    from jupyter_ai_agent_magics import magics
    magics.load_ipython_extension(get_ipython())
    print("✅ Successfully loaded %%agent magic")
except ImportError:
    print("❌ Could not load %%agent magic: jupyter_ai_agent_magics package not installed")
except Exception as e:
    print(f"❌ Error loading %%agent magic: {str(e)}")

# Optional, register autoreload magic
get_ipython().run_line_magic('load_ext', 'autoreload')
'''

# File to save the startup script
script_filename = '99-load-agent-magic.py'
script_path = os.path.join(startup_dir, script_filename)

# Write the script to the startup directory
with open(script_path, 'w') as f:
    f.write(startup_script)

print(f"Created startup script: {script_path}")
print("\nScript content:")
print("="*50)
print(startup_script)
print("="*50)

Created startup script: /Users/alekseyv/.ipython/profile_default/startup/99-load-agent-magic.py

Script content:
# Auto-register %%agent magic
try:
    from jupyter_ai_agent_magics import magics
    magics.load_ipython_extension(get_ipython())
    print("✅ Successfully loaded %%agent magic")
except ImportError:
    print("❌ Could not load %%agent magic: jupyter_ai_agent_magics package not installed")
except Exception as e:
    print(f"❌ Error loading %%agent magic: {str(e)}")



## Verification

The startup script has been created and will be executed automatically the next time you start Jupyter. If you want to test it manually in the current session, you can run the code below:

In [ ]:
try:
    from jupyter_ai_agent_magics import magics
    magics.load_ipython_extension(get_ipython())
    print("✅ Successfully loaded %%agent magic")
except ImportError:
    print("❌ Could not load %%agent magic: jupyter_ai_agent_magics package not installed")
except Exception as e:
    print(f"❌ Error loading %%agent magic: {str(e)}")

## Usage

After restarting Jupyter, you should be able to use the `%%agent` magic in any notebook without having to load it first. For example:

```python
%%agent
What is the capital of France?
```

## Troubleshooting

If the magic doesn't load automatically:

1. Make sure the `jupyter_ai_agent_magics` package is installed:
   ```
   pip install jupyter_ai_agent_magics
   ```

2. Check if the startup script exists at the location printed above. The path depends on your operating system:
   - Windows: Typically `C:\Users\<username>\.ipython\profile_default\startup\99-load-agent-magic.py`
   - macOS/Linux: Typically `~/.ipython/profile_default/startup/99-load-agent-magic.py`

3. Ensure you have the necessary permissions to execute the script.

4. Try restarting the Jupyter server.

5. If the issue persists, you can manually load the magic in each notebook with:
   ```python
   from jupyter_ai_agent_magics import magics
   magics.load_ipython_extension(get_ipython())
   ```

## Note on IPython vs Jupyter Paths

IPython and Jupyter can use different configuration paths. This script uses IPython's built-in functions to locate the correct IPython directory on any platform. The script is intended to work on:

- Windows
- macOS
- Linux

The startup script will be placed in the appropriate location for your operating system.